### Прочитаем train

In [110]:
train = pd.read_csv('train_merged_full_morphed.csv')

In [111]:
train.drop(columns=['fd_slug_id1','fd_slug_id2',\
                    'fd_slug1','fd_slug2','Unnamed: 0' ],axis=1, inplace=True) #07.01.19

train['price'] = train['price']/100

train['morph_description'] = train['morph_description'].fillna('').astype('object')

In [112]:
train.head().T

,0,1,2,3,4
id,3edeb34cf93f490ff760af85,c98febd50dad3cc0ffc86085,ade01e13912a46a99134cc75,ab3e6941c11304c1519aef75,252452a91c944a22c276d995
category,9,22,22,22,22
subcategory,914,2202,2204,2209,2204
name,Сумка DG,Комплект,Пальтишко демисезонное,Attipas,Жилет теплый
description,NaN,8-12 лет,"На девочку 1,5 г,состояние хорошее","Размер-135mm, euro-22.5.Прочная, мягкая, не ск...",NaN
price,1999,350,300,800,500
fd_name1,Женский гардероб,Детский гардероб,Детский гардероб,Детский гардероб,Детский гардероб
fd_name2,Аксессуары,Аксессуары,Верхняя одежда,Обувь,Верхняя одежда
morph_description,,год,девочка год состояние хороший,размер прочный мягкий скользящий обувь маленьк...,
morph_name,сумка dg,комплект,пальтишко демисезонный,attipas,жилет тёплый


In [70]:
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

In [71]:
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion

 Зададим селектор

In [72]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X_, y=None): # 
        return self
    def transform(self, X_):
        return X_.loc[:, self.attribute_names].values # 

In [73]:
class DenseTransformer(BaseEstimator,TransformerMixin):

    def transform(self, X, y=None, **fit_params):
        #return X.todense()
        return X.toarray()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self

Зададим списки столбцов

In [109]:
# получим список столбцов 
full_columns_list = list(train) 

# список числовых категориальных столбцов
num_cat_columns = full_columns_list[1:3] # ['category', 'subcategory']

# целвое значение
target = ['price']
print('num_cat_columns :',num_cat_columns)
print('tfidf_columns   :',tfidf_columns)

other = list(set(full_columns_list) - set(num_cat_columns)  - set(tfidf_columns) - set(target))
print('Остаток         :', other)

num_cat_columns : ['category', 'subcategory']
tfidf_columns   : ['morph_description', 'morph_name']
Остаток         : ['name', 'fd_name2', 'fd_name1', 'id', 'description', 'morph_name_2']


<font color="magenta" size="5">CatBoost.</font><br> 


In [76]:
from catboost import CatBoostClassifier
catbost_model = CatBoostClassifier(depth = 5, num_trees=150, learning_rate = 0.11,verbose=False)

In [131]:
from sklearn.preprocessing import StandardScaler

In [168]:
catbost_model = Pipeline([  
('selector', ColumnSelector(num_cat_columns)), # определим категориальные числовые столбцы: ['category', 'subcategory']
('OHE', OneHotEncoder()),
('denser', DenseTransformer()),
('scaler', StandardScaler()),
('catbost', catbost_model)
])

In [192]:


RND_SEED = 42
param_grid_catb = {
              'depth': [5], 
              'num_trees': [150] , 
               'learning_rate':[0.1,0.12,0.15,0.2]} 
print(param_grid_catb)


{'depth': [5], 'learning_rate': [0.1, 0.12, 0.15, 0.2], 'num_trees': [150]}


In [193]:
from sklearn.model_selection import GridSearchCV

In [194]:
clf_cat = GridSearchCV(catbost_model, param_grid_catb,  
                   cv=4, 
                   scoring='neg_mean_absolute_error', n_jobs=4)

In [195]:
clf_cat.fit(X_train, y_train)

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [ ]:
print(clf_cat.best_params_) # потом надо заменить на clf_1
print(clf_cat.best_score_) # потом надо заменить на clf_1
grid_model_1 = clf_cat.best_estimator_ # потом надо заменить на clf_1

In [133]:
y = train['price']
X_train = train.iloc[:1000000,:]
y_train = y.iloc[:1000000]

X_test = train.iloc[1000000:,:]
y_test = y.iloc[1000000:]

In [134]:
print(X_train.shape)
print(y_train.shape)

(1000000, 10)
(1000000,)


In [135]:
%%time 
model.fit(X_train, y_train)

/home/vladimirpolanski6/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


CPU times: user 7min 5s, sys: 35.8 s, total: 7min 41s
Wall time: 1min 41s


Pipeline(memory=None,
     steps=[('selector', ColumnSelector(attribute_names=['category', 'subcategory'])), ('OHE', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values='auto', sparse=True)), ('denser', DenseTransformer()), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('catbost', <catboost.core.CatBoostClassifier object at 0x7f0e58b6a5c0>)])

In [137]:
y_hat = model.predict(X_test)
y_hat.shape

(748890,)

In [90]:
#y_hat = np.exp(y_hat_log)-1

In [91]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error 

In [138]:
print(mean_absolute_error(y_test,y_hat))
#print(mean_squared_log_error(y_test,y_hat))

317706.3153451507


In [139]:
y_test_df = pd.DataFrame(y_test)
y_hat_df = pd.DataFrame(y_hat, index=range(1000000,len(y_hat)+1000000))

In [140]:
print(y_hat_df.shape)
print(y_test_df.shape)

(748890, 1)
(748890, 1)


In [141]:
y_test_df.head()

,price
1000000,3000.0
1000001,120.0
1000002,500.0
1000003,12000.0
1000004,0.0


In [142]:
y_hat_df.head()

,0
1000000,1.0
1000001,1.0
1000002,1.0
1000003,1.0
1000004,1.0


In [ ]:
# объединим предсказанную часть трейна, цену и предсказанную цену
train_predicted = pd.concat([train__, y_hat_df], axis=1)

In [ ]:
train_predicted.head(50)

In [ ]:
train_predicted.shape

### Попробуем по частям выполнить действия

In [113]:
getmatrix = Pipeline([  
('selector', ColumnSelector(num_cat_columns)), # определим категориальные числовые столбцы: ['category', 'subcategory']
('OHE', OneHotEncoder()),
        
('denser',DenseTransformer())])

In [116]:
matrix = getmatrix.fit_transform(train) # Получим  OHE матрицу и сделаем из нее плотную

/home/vladimirpolanski6/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [118]:
matrix_df = pd.DataFrame(matrix) # переведем ее в датафрейм
matrix_df

,0,1,2,3,4,5,6,7,8,9,...,262,263,264,265,266,267,268,269,270,271
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [121]:
y

0            1999.0
1             350.0
2             300.0
3             800.0
4             500.0
5             300.0
6            1500.0
7            2000.0
8             350.0
9            3500.0
10           4000.0
11           6000.0
12            300.0
13            800.0
14            500.0
15           2500.0
16            990.0
17            500.0
18            900.0
19           1500.0
20           1000.0
21          25990.0
22            800.0
23           1300.0
24           1000.0
25            500.0
26            400.0
27            150.0
28           1000.0
29            990.0
30            800.0
31            400.0
32            350.0
33           1200.0
34            900.0
35            280.0
36           1500.0
37           6500.0
38           4900.0
39            650.0
40           2600.0
41            300.0
42            800.0
43            900.0
44            250.0
45            500.0
46            300.0
47           1000.0
48            400.0
49            100.0


In [124]:
catbost_model.fit(matrix_df.iloc[:1000000,:], y.iloc[:1000000])

In [126]:
y_hat = catbost_model.predict(matrix_df.iloc[1000000:,:])

In [130]:
y_hat_df = pd.DataFrame(y_hat, index=range(1000000,len(y_hat)+1000000))
y_hat_df

,0
1000000,1.0
1000001,1.0
1000002,1.0
1000003,1.0
1000004,1.0
1000005,1.0
1000006,1.0
1000007,1.0
1000008,1.0
1000009,1.0


### Линейная регрессия

In [143]:
from sklearn.linear_model import LinearRegression

In [148]:
model2 = Pipeline([  
('selector', ColumnSelector(num_cat_columns)), # определим категориальные числовые столбцы: ['category', 'subcategory']
('OHE', OneHotEncoder()),
('denser', DenseTransformer()),
('scaler', StandardScaler()),
('lr', LinearRegression())
])

In [154]:
# Другая версия param_grid, в ней закомтен scaler__with_mean, т.к. в .get_params().keys() его (???) нет
RND_SEED = 42
param_grid = {
    'scaler__with_mean': [False, True],
    'lr__normalize': [False, True],
    'clf__random_state': [RND_SEED],
}

print(param_grid)


{'scaler__with_mean': [False, True], 'lr__normalize': [False, True], 'clf__random_state': [42]}


In [155]:
%%time 
model2.fit(X_train, y_train)

/home/vladimirpolanski6/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


CPU times: user 47.3 s, sys: 16.4 s, total: 1min 3s
Wall time: 31.2 s


Pipeline(memory=None,
     steps=[('selector', ColumnSelector(attribute_names=['category', 'subcategory'])), ('OHE', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values='auto', sparse=True)), ('denser', DenseTransformer()), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('lr', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False))])

In [156]:
y_hat = model2.predict(X_test)
y_hat.shape

(748890,)

In [157]:
y_test_df = pd.DataFrame(y_test)
y_hat_df = pd.DataFrame(y_hat, index=range(1000000,len(y_hat)+1000000))

In [158]:
print(y_hat_df.shape)
print(y_test_df.shape)

(748890, 1)
(748890, 1)


In [159]:
y_test_df.head()

,price
1000000,3000.0
1000001,120.0
1000002,500.0
1000003,12000.0
1000004,0.0


In [160]:
y_hat_df.head()

,0
1000000,931072.57938
1000001,518.57938
1000002,495.82938
1000003,5362.57938
1000004,3752.57938


In [161]:
print(mean_absolute_error(y_test_df,y_hat_df))


210008914035.00006


In [163]:
from sklearn.linear_model import ElasticNet

In [164]:
regressor = ElasticNet(alpha=1.0, l1_ratio=0.5) #alpha = 1.0

In [165]:
model3 = Pipeline([  
('selector', ColumnSelector(num_cat_columns)), # определим категориальные числовые столбцы: ['category', 'subcategory']
('OHE', OneHotEncoder(sparse=False)),
('scaler', StandardScaler()),
('lr', LinearRegression())
])

In [166]:
def rlmse_score(y_test, y_hat):
    
    assert len(y_test) == len(y_hat) # !!!!!
    rlmse = np.sqrt(np.mean((np.log(y_test+1) - np.log(y_hat+1))**2))
    
    return rlmse

# Эту функцию трогать не надо
def rlmse_scorer(estimator, X, y):
    y_hat = estimator.predict(X)
    
    return rlmse_score(y, y_hat)